## Training attention based model

In [1]:
%matplotlib inline

import sys
sys.path.append("../../")
import tensorflow as tf
from train_model import train

from sklearn.linear_model import LinearRegression
from yellowbrick.regressor import ResidualsPlot, PredictionError

import seaborn as sns
import matplotlib.pyplot as plt

from matplotlib import rcParams
rcParams['figure.figsize'] = (8, 4)
rcParams['figure.dpi'] = 100
rcParams['font.size'] = 8
rcParams['font.family'] = 'sans-serif'
rcParams['axes.facecolor'] = '#ffffff'
rcParams['lines.linewidth'] = 2.0

In [3]:
import os
file_path = r"D:\Final_file\ASI-main\output\models\BJ\asi_BJ_weights.hdf5"
from tensorflow.keras import backend as K
K.clear_session()

if os.path.exists(file_path):
    os.remove(file_path)
    print(f"Removed the file: {file_path}")
else:
    print(f"The file {file_path} does not exist")

The file D:\Final_file\ASI-main\output\models\BJ\asi_BJ_weights.hdf5 does not exist


In [4]:
##BEST IT

hyperparameter={
"num_nearest":30,
"sigma":10,
"geointerpolation": 'asi',
'type_compat_funct_eucli':'kernel_gaussiano',
'Num_heads':4,
"learning_rate":0.001,
"batch_size":250,
"num_neuron":60,
"num_layers":3,
"size_embedded":60,
"num_nearest_geo":15,
"num_nearest_eucli":15,
"id_dataset":'BJ',
"epochs":300,
"optimier":'adam',
"validation_split":0.1,
"label":'asi_BJ',
"early_stopping": False,
'scale_log':False,
"graph_label":'matrix',
}

In [5]:
from tensorflow.keras import backend as K
K.clear_session()

In [6]:
spatial = train(**hyperparameter)

In [7]:
dataset,\
result,\
fit,\
embedded_train,\
embedded_test,\
predict_regression_train,\
predict_regression_test = spatial()

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')
Epoch 1/200
83/83 [==============================] - 1s 4ms/step - loss: 65284.0781 - root_mean_squared_error: 70048.0781 - val_loss: 59539.3711 - val_root_mean_squared_error: 64581.1953

Epoch 00001: val_loss improved from inf to 59539.37109, saving model to d:\Final_file\ASI-main/output/models/BJ\asi_BJ_weights.hdf5
Epoch 2/200
83/83 [==============================] - 0s 3ms/step - loss: 32875.2812 - root_mean_squared_error: 41240.4844 - val_loss: 21025.9336 - val_root_mean_squared_error: 27336.5020

Epoch 00002: val_loss improved from 59539.37109 to 21025.93359, saving model to d:\Final_file\ASI-main/output/models/BJ\asi_BJ_weights.hdf5
Epoch 3/200
83/83 [==============================] - 0s 3ms/step - loss: 14610.5342 - root_mean_squared_error: 20035.4863 - val_loss: 10617.7773 - val_root_mean_squared_error: 14839.

In [8]:
#asi

print('################# Test ##########################')
print('MALE test:.... {}'.format(result[0]))
print('RMSE test:.... {}'.format(result[1]))
print('MAPE test:.... {}'.format(result[2]))
print('################# Train ##########################')
print('MALE train:.... {}'.format(result[3]))
print('RMSE train:.... {}'.format(result[4]))
print('MAPE train:.... {}'.format(result[5]))

################# Test ##########################
MALE test:.... 0.07503143544221215
RMSE test:.... 7934.296784356182
MAPE test:.... 7.7603797104294046
################# Train ##########################
MALE train:.... 0.07503143544221215
RMSE train:.... 7924.862042089876
MAPE train:.... 7.332779247367394


In [7]:
#asi multi

print('################# Test ##########################')
print('MALE test:.... {}'.format(result[0]))
print('RMSE test:.... {}'.format(result[1]))
print('MAPE test:.... {}'.format(result[2]))
print('################# Train ##########################')
print('MALE train:.... {}'.format(result[3]))
print('RMSE train:.... {}'.format(result[4]))
print('MAPE train:.... {}'.format(result[5]))

################# Test ##########################
MALE test:.... 0.07305498744351963
RMSE test:.... 7797.654386494778
MAPE test:.... 7.55216402123668
################# Train ##########################
MALE train:.... 0.07305498744351963
RMSE train:.... 7803.833131146778
MAPE train:.... 7.283034600113412


In [10]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

In [6]:
path = r'D:\Final_file\ASI-main\datasets\BJ\data.npz'
data = np.load(path)

In [7]:
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Base models benchmarking 

In [9]:

# Define the mean absolute percentage error
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


def mean_absolute_log_error(y_true, y_pred):
    # Ensure all values are positive with a small constant (e.g., 1e-10)
    y_true_pos = np.maximum(y_true, 1e-10) + 1
    y_pred_pos = np.maximum(y_pred, 1e-10) + 1
    y_true_log = np.log(y_true_pos)
    y_pred_log = np.log(y_pred_pos)
    return mean_absolute_error(y_true_log, y_pred_log)



# Initialize lists to store test set evaluation metrics for each model
test_mae_list = {}
test_mse_list = {}
test_rmse_list = {}
test_r2_list = {}
test_mape_list = {}
test_male_list = {} 
# Define the models and their respective parameter grids
models = {
    'Linear Regression': {
        'model': LinearRegression(),
        'params': {}
    },
    'KNN': {
        'model': KNeighborsRegressor(),
        'params': {'n_neighbors': [5, 10, 15, 20]}
    },
    'Decision Tree': {
        'model': DecisionTreeRegressor(),
        'params': {'max_depth': [5, 9, 12, 15]}
    },
    'Random Forest': {
        'model': RandomForestRegressor(),
        'params': {'n_estimators': [50, 100, 150], 'max_depth': [8, 12, 16]}
    },
    'SVM': {
        'model': SVR(),
        'params': {'C': [10, 100], 'gamma': ['scale', 'auto']}
    },
    'LightGBM': {
        'model': LGBMRegressor(),
        'params': {'n_estimators': [200,400,1000], 'learning_rate': [0.05]}
    },
    'CatBoost': {
        'model': CatBoostRegressor(verbose=0),
        'params': {'depth': [ 8, 10], 'learning_rate': [0.05],'n_estimators': [200,400,1000]}
    },
    'XGBoost': {
        'model': XGBRegressor( learning_rate=0.05, random_state=42),
        'params': {'max_depth': [5, 7, 9], 'learning_rate': [0.05],'n_estimators': [200,400,1000]}
    }
}

# Define the K-fold cross-validator
cv = KFold(n_splits=10, shuffle=True, random_state=42)

# For each model
for name, model_info in models.items():
    test_mae_list[name] = []
    test_mse_list[name] = []
    test_rmse_list[name] = []
    test_r2_list[name] = []
    test_mape_list[name] = []
    test_male_list[name] = [] 

    for train_idx, val_idx in cv.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

        # Grid Search for hyperparameter tuning
        grid = GridSearchCV(estimator=model_info['model'], param_grid=model_info['params'], cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
        grid_result = grid.fit(X_train_fold, y_train_fold)
        best_model = grid_result.best_estimator_

        # Make predictions on the test set
        y_pred_test = best_model.predict(X_test)

        # Evaluate the model on the test set
        test_mae = mean_absolute_error((y_test), (y_pred_test))
        test_mse = mean_squared_error((y_test), (y_pred_test))
        test_rmse = np.sqrt(test_mse)
        test_r2 = r2_score((y_test), (y_pred_test))
        test_mape = mean_absolute_percentage_error((y_test), (y_pred_test))
        test_male = mean_absolute_log_error((y_test), (y_pred_test)) 

        # Store the test set metrics in the lists
        test_mae_list[name].append(test_mae)
        test_mse_list[name].append(test_mse)
        test_rmse_list[name].append(test_rmse)
        test_r2_list[name].append(test_r2)
        test_mape_list[name].append(test_mape)
        test_male_list[name].append(test_male)

    # Calculate the average and best metrics for the test set
    avg_test_mae = np.mean(test_mae_list[name])
    avg_test_mse = np.mean(test_mse_list[name])
    avg_test_rmse = np.mean(test_rmse_list[name])
    avg_test_r2 = np.mean(test_r2_list[name])
    avg_test_mape = np.mean(test_mape_list[name])
    avg_test_male = np.mean(test_male_list[name])

    best_test_mae = np.min(test_mae_list[name])
    best_test_mse = np.min(test_mse_list[name])
    best_test_rmse = np.min(test_rmse_list[name])
    best_test_r2 = np.max(test_r2_list[name])
    best_test_mape = np.min(test_mape_list[name])
    best_test_male = np.min(test_male_list[name])

    print(f"Test Set Evaluation for {name}")
    print(f"Average Test MAE: {avg_test_mae}, Best Test MAE: {best_test_mae}")
    print(f"Average Test MSE: {avg_test_mse}, Best Test MSE: {best_test_mse}")
    print(f"Average Test RMSE: {avg_test_rmse}, Best Test RMSE: {best_test_rmse}")
    print(f"Average Test R2: {avg_test_r2}, Best Test R2: {best_test_r2}")
    print(f"Average Test MAPE: {avg_test_mape}, Best Test MAPE: {best_test_mape}")
    print(f"Average Test MALE: {avg_test_male}, Best Test MALE: {best_test_male}")
    print("\n")



Test Set Evaluation for Linear Regression
Average Test MAE: 15548.845706714788, Best Test MAE: 15543.97607945616
Average Test MSE: 422647058.7583653, Best Test MSE: 422345049.00986516
Average Test RMSE: 20558.381309287437, Best Test RMSE: 20551.035229639045
Average Test R2: 0.3759393487396898, Best Test R2: 0.37638528204605104
Average Test MAPE: 25.576186978287872, Best Test MAPE: 25.555329581769943
Average Test MALE: 0.23955034569615022, Best Test MALE: 0.23941003702646574


Test Set Evaluation for KNN
Average Test MAE: 7524.034346760071, Best Test MAE: 7481.2034676007015
Average Test MSE: 147594201.3400785, Best Test MSE: 146236952.14427322
Average Test RMSE: 12148.77489051982, Best Test RMSE: 12092.847147974426
Average Test R2: 0.782069385077173, Best Test R2: 0.7840734350273728
Average Test MAPE: 11.816468504276283, Best Test MAPE: 11.746265965009945
Average Test MALE: 0.11213164656988783, Best Test MALE: 0.11155264854862162


Test Set Evaluation for Decision Tree
Average Test MAE:

## Testing the embeddings

In [11]:
X_train ,X_test, y_train , y_test = embedded_train,embedded_test,dataset.y_train,dataset.y_test


In [12]:

# Define the mean absolute percentage error
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


def mean_absolute_log_error(y_true, y_pred):
    # Ensure all values are positive with a small constant (e.g., 1e-10)
    y_true_pos = np.maximum(y_true, 1e-10) + 1
    y_pred_pos = np.maximum(y_pred, 1e-10) + 1
    y_true_log = np.log(y_true_pos)
    y_pred_log = np.log(y_pred_pos)
    return mean_absolute_error(y_true_log, y_pred_log)



# Initialize lists to store test set evaluation metrics for each model
test_mae_list = {}
test_mse_list = {}
test_rmse_list = {}
test_r2_list = {}
test_mape_list = {}
test_male_list = {} 
# Define the models and their respective parameter grids
models = {
    'Linear Regression': {
        'model': LinearRegression(),
        'params': {}
    },
    'KNN': {
        'model': KNeighborsRegressor(),
        'params': {'n_neighbors': [5, 10, 15, 20]}
    },
    'Decision Tree': {
        'model': DecisionTreeRegressor(),
        'params': {'max_depth': [5, 9, 12, 15]}
    },
    'Random Forest': {
        'model': RandomForestRegressor(),
        'params': {'n_estimators': [50, 100, 150], 'max_depth': [8, 12, 16]}
    },
    'SVM': {
        'model': SVR(),
        'params': {'C': [10, 100], 'gamma': ['scale', 'auto']}
    },
    'LightGBM': {
        'model': LGBMRegressor(),
        'params': {'n_estimators': [200,400,1000], 'learning_rate': [0.05]}
    },
    'CatBoost': {
        'model': CatBoostRegressor(verbose=0),
        'params': {'depth': [ 8, 10], 'learning_rate': [0.05],'n_estimators': [200,400,1000]}
    },
    'XGBoost': {
        'model': XGBRegressor( learning_rate=0.05, random_state=42),
        'params': {'max_depth': [5, 7, 9], 'learning_rate': [0.05],'n_estimators': [200,400,1000]}
    }
}

# Define the K-fold cross-validator
cv = KFold(n_splits=10, shuffle=True, random_state=42)

# For each model
for name, model_info in models.items():
    test_mae_list[name] = []
    test_mse_list[name] = []
    test_rmse_list[name] = []
    test_r2_list[name] = []
    test_mape_list[name] = []
    test_male_list[name] = [] 

    for train_idx, val_idx in cv.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

        # Grid Search for hyperparameter tuning
        grid = GridSearchCV(estimator=model_info['model'], param_grid=model_info['params'], cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
        grid_result = grid.fit(X_train_fold, y_train_fold)
        best_model = grid_result.best_estimator_

        # Make predictions on the test set
        y_pred_test = best_model.predict(X_test)

        # Evaluate the model on the test set
        test_mae = mean_absolute_error((y_test), (y_pred_test))
        test_mse = mean_squared_error((y_test), (y_pred_test))
        test_rmse = np.sqrt(test_mse)
        test_r2 = r2_score((y_test), (y_pred_test))
        test_mape = mean_absolute_percentage_error((y_test), (y_pred_test))
        test_male = mean_absolute_log_error((y_test), (y_pred_test)) 

        # Store the test set metrics in the lists
        test_mae_list[name].append(test_mae)
        test_mse_list[name].append(test_mse)
        test_rmse_list[name].append(test_rmse)
        test_r2_list[name].append(test_r2)
        test_mape_list[name].append(test_mape)
        test_male_list[name].append(test_male)

    # Calculate the average and best metrics for the test set
    avg_test_mae = np.mean(test_mae_list[name])
    avg_test_mse = np.mean(test_mse_list[name])
    avg_test_rmse = np.mean(test_rmse_list[name])
    avg_test_r2 = np.mean(test_r2_list[name])
    avg_test_mape = np.mean(test_mape_list[name])
    avg_test_male = np.mean(test_male_list[name])

    best_test_mae = np.min(test_mae_list[name])
    best_test_mse = np.min(test_mse_list[name])
    best_test_rmse = np.min(test_rmse_list[name])
    best_test_r2 = np.max(test_r2_list[name])
    best_test_mape = np.min(test_mape_list[name])
    best_test_male = np.min(test_male_list[name])

    print(f"Test Set Evaluation for {name}")
    print(f"Average Test MAE: {avg_test_mae}, Best Test MAE: {best_test_mae}")
    print(f"Average Test MSE: {avg_test_mse}, Best Test MSE: {best_test_mse}")
    print(f"Average Test RMSE: {avg_test_rmse}, Best Test RMSE: {best_test_rmse}")
    print(f"Average Test R2: {avg_test_r2}, Best Test R2: {best_test_r2}")
    print(f"Average Test MAPE: {avg_test_mape}, Best Test MAPE: {best_test_mape}")
    print(f"Average Test MALE: {avg_test_male}, Best Test MALE: {best_test_male}")
    print("\n")



Test Set Evaluation for Linear Regression
Average Test MAE: 4737.711266965848, Best Test MAE: 4730.929100536339
Average Test MSE: 60630159.23508022, Best Test MSE: 60523234.02131094
Average Test RMSE: 7786.535095030396, Best Test RMSE: 7779.667989143942
Average Test R2: 0.9104763753250381, Best Test R2: 0.9106342560369897
Average Test MAPE: 7.6150270799643724, Best Test MAPE: 7.5946835800486205
Average Test MALE: 0.07334861525694093, Best Test MALE: 0.07320602472883267


Test Set Evaluation for KNN
Average Test MAE: 4985.339389667251, Best Test MAE: 4964.006943957968
Average Test MSE: 64241577.28195085, Best Test MSE: 63696009.10482006
Average Test RMSE: 8015.06327789595, Best Test RMSE: 7980.977954162012
Average Test R2: 0.9051439262955233, Best Test R2: 0.9059494864546958
Average Test MAPE: 7.99396150569996, Best Test MAPE: 7.958586603988582
Average Test MALE: 0.07705106468015747, Best Test MALE: 0.07675223418344448


Test Set Evaluation for Decision Tree
Average Test MAE: 4872.05280